In [1]:
# !pip install datasets transformers pandasql beautifultable -qqq
# torch.cuda.empty_cache()

In [2]:
import pandas as pd
import re

In [3]:
model_checkpoint = "t5-small"
model_date = '2021-10-17`'
prefix = "translate english to sparql: "
model_name = "yazdipour/text-to-sparql-t5-small-2021-10-18_12-12"

In [4]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, T5TokenizerFast
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

In [5]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = T5TokenizerFast.from_pretrained(model_name)
tokenizer = T5TokenizerFast.from_pretrained(model_checkpoint, src_lang="en", tgt_lang="sparql")

In [6]:
translator = pipeline(
    "translation_xx_to_yy",
    model=model,
    tokenizer=tokenizer,
    device=0 #0 for cuda, -1 for cpu
)

In [7]:
def replace_all(text, dict):
    for i, j in dict.items():
        text = text.replace(i, j)
    return text

In [8]:
kdwd_date = '2021-10-11'
df_q = pd.read_csv(f'../../../data/kdwd/{kdwd_date}/q.csv')
df_p = pd.read_csv(f'../../../data/kdwd/{kdwd_date}/p.csv')

In [57]:
def findID(pref, x):
    word = x.replace("_", " ").lower()
    try:
        if 'wd' != pref:
            # s = df_p[df_p.en.str.contains(word, na=False)]
            s = df_p[df_p['en']==word]
            if len(s)>0:
                id = s.iloc[0]['id']
                return 'P'+str(id)
        # Use else to do not replace missing Ps with Qs
        s = df_q[df_q['en']==word]
        # s = df_q[df_q.en.str.contains(word, na=False)]
        id = s.iloc[0]['id']
        return 'Q'+str(id)
    except: return x

In [10]:
findID('wd', 'instance')

'Q1664689'

In [11]:
def decode_props(qry):
    rep_dict = {}
    qry = qry.replace("[", "{").replace("]", "}")
    for m in re.finditer(":\w+", qry):
        pref = qry[m.start(0) - 3 : m.start(0)].strip()
        rep_dict[m.group(0)[1:]] = findID(pref, m.group(0)[1:])
    return replace_all(qry, rep_dict)


In [12]:
decode_props('SELECT ?answer WHERE [ wd:berlin wdt:instance_of ?answer . ?answer wdt:instance_of wd:mayor ]')

'SELECT ?answer WHERE { wd:Q64 wdt:instance_of ?answer . ?answer wdt:instance_of wd:Q30185 }'

In [13]:
def translate(q):
    tr = translator(prefix+q, max_length=100)[0]['translation_text']
    return (tr, decode_props(tr))

In [14]:
from beautifultable import BeautifulTable
def pretty_translate(t,q): 
    ans = translate(q)
    # print('QUESTION:\n' + q + '\nTARGET:\n'+ t +'\nRESULT:\n' + ans[0]+ '\nRESULT-DECODED:\n' + ans[1])
    table = BeautifulTable(maxwidth=140)
    # table.column_headers = ["", "Man Utd","Man City","T Hotspur"]
    table.rows.append(['QUESTION', q])
    table.rows.append(['Target', t])
    table.rows.append(['RawRESULT', ans[0]])
    table.rows.append(['RESULT', ans[1]])
    print(table)

# TESTING

In [15]:
prefix

'translate english to sparql: '

In [16]:
translator(prefix+'Who is Marlin Manson?', max_length=100)

[{'translation_text': 'select distinct ?obj where [ wd:marlin_manson wdt:head_of_work ?obj . ?obj wdt:instance_of wd:human ]'}]

In [58]:
# <re.Match object; span=(52, 66), match=':marlin_manson'>
# ('select distinct ?sbj where { ?sbj wdt:instance_of wd:marlin_manson . ?sbj wdt:instance_of wd:human }',
#  'select distinct ?sbj where { ?sbj wdt:P31 wd:marlin_manson . ?sbj wdt:P31 wd:Q5 }')
translate('Who is Marlin Manson?')

('select distinct ?obj where [ wd:marlin_manson wdt:head_of_work ?obj . ?obj wdt:instance_of wd:human ]',
 'select distinct ?obj where { wd:marlin_manson wdt:Q1592254 ?obj . ?obj wdt:P31 wd:Q5 }')

In [59]:
translate('How old is Bill Gates?')

("select ?value where [ wd:bill_gates p:date_of_birth ?s . ?s ps:date_of_birth ?x filter(contains(YEAR(?x),'1980')) . ?s pq:point_in_time ?value]",
 "select ?value where { wd:Q5284 p:P569 ?s . ?s ps:P569 ?x filter(contains(YEAR(?x),'1980')) . ?s pq:P585 ?value}")

In [60]:
pretty_translate('select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }','Who is the country for head of state of Mahmoud Abbas?')

+-----------+---------------------------------------------------------------------------------------------------------+
| QUESTION  |                         Who is the country for head of state of Mahmoud Abbas?                          |
+-----------+---------------------------------------------------------------------------------------------------------+
|  Target   |             select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }              |
+-----------+---------------------------------------------------------------------------------------------------------+
| RawRESULT | select distinct ?obj where [ wd:mamoud_abbas wdt:head_of_state ?obj . ?obj wdt:instance_of wd:country ] |
+-----------+---------------------------------------------------------------------------------------------------------+
|  RESULT   |           select distinct ?obj where { wd:mamoud_abbas wdt:P35 ?obj . ?obj wdt:P31 wd:Q6256 }           |
+-----------+---------------------------

In [61]:
pretty_translate('select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }',"What country is Mahmoud Abbas the head of state of?")

+-----------+---------------------------------------------------------------------------------------------------------+
| QUESTION  |                           What country is Mahmoud Abbas the head of state of?                           |
+-----------+---------------------------------------------------------------------------------------------------------+
|  Target   |             select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }              |
+-----------+---------------------------------------------------------------------------------------------------------+
| RawRESULT | select distinct ?obj where [ wd:mamoud_abbas wdt:head_of_state ?obj . ?obj wdt:instance_of wd:country ] |
+-----------+---------------------------------------------------------------------------------------------------------+
|  RESULT   |           select distinct ?obj where { wd:mamoud_abbas wdt:P35 ?obj . ?obj wdt:P31 wd:Q6256 }           |
+-----------+---------------------------

In [62]:
pretty_translate('select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }','Who is the country for head of state of Mahmoud Abbas?')

+-----------+---------------------------------------------------------------------------------------------------------+
| QUESTION  |                         Who is the country for head of state of Mahmoud Abbas?                          |
+-----------+---------------------------------------------------------------------------------------------------------+
|  Target   |             select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }              |
+-----------+---------------------------------------------------------------------------------------------------------+
| RawRESULT | select distinct ?obj where [ wd:mamoud_abbas wdt:head_of_state ?obj . ?obj wdt:instance_of wd:country ] |
+-----------+---------------------------------------------------------------------------------------------------------+
|  RESULT   |           select distinct ?obj where { wd:mamoud_abbas wdt:P35 ?obj . ?obj wdt:P31 wd:Q6256 }           |
+-----------+---------------------------

In [63]:
pretty_translate("SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}","Which female actress is the voice over on South Park and is employed as a singer?".lower())

+-----------+--------------------------------------------------------------------------------------------------------------+
| QUESTION  |              which female actress is the voice over on south park and is employed as a singer?               |
+-----------+--------------------------------------------------------------------------------------------------------------+
|  Target   |               SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}               |
+-----------+--------------------------------------------------------------------------------------------------------------+
| RawRESULT | select ?answer where [ wd:south_park wdt:voice_over ?answer . ?answer wdt:position_held wd:single_and_song ] |
+-----------+--------------------------------------------------------------------------------------------------------------+
|  RESULT   |        select ?answer where { wd:Q16538 wdt:Q12280274 ?answer . ?answer wdt:P39 wd:single_and_song }         |


In [64]:
pretty_translate("SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}","Which female actress on South Park is the voice over and is used as a singer?")

+-----------+-------------------------------------------------------------------------------------------------+
| QUESTION  |          Which female actress on South Park is the voice over and is used as a singer?          |
+-----------+-------------------------------------------------------------------------------------------------+
|  Target   |        SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}         |
+-----------+-------------------------------------------------------------------------------------------------+
| RawRESULT | select ?answer where [ wd:south_park wdt:voice_over ?answer . ?answer wdt:voice_over wd:single] |
+-----------+-------------------------------------------------------------------------------------------------+
|  RESULT   |   select ?answer where { wd:Q16538 wdt:Q12280274 ?answer . ?answer wdt:Q12280274 wd:Q134556}    |
+-----------+-------------------------------------------------------------------------------------------

In [65]:
# Paul Erdős (Q173746)
pretty_translate("select distinct ?answer where { wd:Q173746 wdt:P3973 ?answer}","Which is the PIM authority ID of Paul Erd?")

+-----------+--------------------------------------------------------------------------------------+
| QUESTION  |                      Which is the PIM authority ID of Paul Erd?                      |
+-----------+--------------------------------------------------------------------------------------+
|  Target   |            select distinct ?answer where { wd:Q173746 wdt:P3973 ?answer}             |
+-----------+--------------------------------------------------------------------------------------+
| RawRESULT | select ?answer where [ wd:paul_ölt wdt:authority_id ?X . ?X wdt:named_after ?answer] |
+-----------+--------------------------------------------------------------------------------------+
|  RESULT   |    select ?answer where { wd:paul_ölt wdt:authority_id ?X . ?X wdt:P138 ?answer}     |
+-----------+--------------------------------------------------------------------------------------+


In [66]:
pretty_translate("SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),'2009')) }", 
            "What was the population of Somalia in 2009-0-0?")

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                       What was the population of Somalia in 2009-0-0?                                        |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   |        SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),'2009')) }        |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT | select ?obj where [ wd:samala p:population ?s . ?s ps:population ?obj . ?s pq:point_in_time ?x filter(contains(YEAR(?x),'200 |
|           |                                                            9')) ]                                                            |
+-----------+

In [67]:
translate('Humans born in New York City') #random query - answer seems correct

('select distinct ?obj where [ wd:new_york_city wdt:born_of ?obj . ?obj wdt:instance_of wd:human ]',
 'select distinct ?obj where { wd:Q60 wdt:born_of ?obj . ?obj wdt:P31 wd:Q5 }')

In [68]:
# From QALD
target = "ASK WHERE { <http://dbpedia.org/resource/Taiko> a <http://dbpedia.org/class/yago/WikicatJapaneseMusicalInstruments> }"
q =  "Are Taiko some kind of Japanese musical instrument?"
pretty_translate(target, q)

+-----------+-----------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                  Are Taiko some kind of Japanese musical instrument?                                  |
+-----------+-----------------------------------------------------------------------------------------------------------------------+
|  Target   | ASK WHERE { <http://dbpedia.org/resource/Taiko> a <http://dbpedia.org/class/yago/WikicatJapaneseMusicalInstruments> } |
+-----------+-----------------------------------------------------------------------------------------------------------------------+
| RawRESULT |                               ask where [ wd:taiko wdt:musical_instrument wd:japanese ]                               |
+-----------+-----------------------------------------------------------------------------------------------------------------------+
|  RESULT   |                                      ask where {

In [69]:
# From QALD
target = "PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT DISTINCT ?uri WHERE { ?uri dct:subject dbc:Assassins_of_Julius_Caesar }"
q =  "Who killed Caesar?"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                                      Who killed Caesar?                                                      |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT DISTINCT ?uri WHERE { ?ur |
|           |                                        i dct:subject dbc:Assassins_of_Julius_Caesar }                                        |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |               select distinct ?obj where [ wd:cescar wdt:place_of_death ?obj . ?obj wdt:instance_of wd:human ]               |
+-----------+

In [70]:
# From QALD
q = 'What is the highest mountain in Germany?'
target = "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX onto: <http://dbpedia.org/ontology/>  \nSELECT ?uri WHERE { ?uri rdf:type onto:Mountain ; onto:elevation ?elevation ; onto:locatedInArea <http://dbpedia.org/resource/Germany> } ORDER BY DESC(?elevation) LIMIT 1"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                           What is the highest mountain in Germany?                                           |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX onto:  |
|           |                                               <http://dbpedia.org/ontology/>                                                 |
|           | SELECT ?uri WHERE { ?uri rdf:type onto:Mountain ; onto:elevation ?elevation ; onto:locatedInArea <http://dbpedia.org/resourc |
|           |                                        e/Germany> } ORDER BY DESC(?elevation) LIMIT 1                                        |
+-----------+

In [71]:
# From QALD
q = 'Which American presidents were in office during the Vietnam War?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT ?uri WHERE { ?uri dct:subject dbc:Presidents_of_the_United_States . res:Vietnam_War dbo:commander ?uri }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                               Which American presidents were in office during the Vietnam War?                               |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX dct: <http://purl.org/dc/terms/ |
|           | > PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT ?uri WHERE { ?uri dct:subject dbc:Presidents_of_the_United_Stat |
|           |                                          es . res:Vietnam_War dbo:commander ?uri }                                           |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |

In [72]:
# From QALD
q = 'How many gold medals did Michael Phelps win at the 2008 Olympics?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT Count(?sub) as ?c WHERE { ?sub dbo:goldMedalist dbr:Michael_Phelps . filter (contains (str(?sub), \"2008\") && contains (str(?sub), \"Olympics\")) }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                              How many gold medals did Michael Phelps win at the 2008 Olympics?                               |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/02 |
|           | /22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT Count(?sub) as ?c WHERE { ?sub dbo:goldMedal |
|           |             ist dbr:Michael_Phelps . filter (contains (str(?sub), "2008") && contains (str(?sub), "Olympics")) }             |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |

In [73]:
# From QALD
q = 'What is the profession of Frank Herbert?'
target = "PREFIX dbpedia2: <http://dbpedia.org/property/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?string WHERE { res:Frank_Herbert dbpedia2:occupation ?string }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                           What is the profession of Frank Herbert?                                           |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | PREFIX dbpedia2: <http://dbpedia.org/property/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?string WHERE { r |
|           |                                        es:Frank_Herbert dbpedia2:occupation ?string }                                        |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |          select distinct ?obj where [ wd:frank_herrand wdt:profession ?obj . ?obj wdt:instance_of wd:professional ]          |
+-----------+

In [74]:
# From QALD
q = 'How many seats does the home stadium of FC Porto have?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbp: <http://dbpedia.org/property/> PREFIX dbr: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX db: <http://dbpedia.org/> SELECT ?capacity WHERE { { dbr:FC_Porto dbo:ground ?ground . ?ground dbo:capacity ?capacity } UNION { dbr:FC_Porto dbo:ground ?ground . ?ground dbp:capacity ?capacity } }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                    How many seats does the home stadium of FC Porto have?                                    |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbp: <http://dbpedia.org/property/> PREFIX dbr: <http://dbpedia.org/resour |
|           | ce/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX d |
|           | b: <http://dbpedia.org/> SELECT ?capacity WHERE { { dbr:FC_Porto dbo:ground ?ground . ?ground dbo:capacity ?capacity } UNION |
|           |                            { dbr:FC_Porto dbo:ground ?ground . ?ground dbp:capacity ?capacity } }                            |
+-----------+

In [75]:
# From QALD
q = 'Which frequent flyer program has the most airlines?'
target = "SELECT ?uri WHERE { ?airline <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Airline> . ?airline <http://dbpedia.org/property/frequentFlyer> ?uri. } GROUP BY ?uri ORDER BY DESC(COUNT(DISTINCT ?airline)) OFFSET 0 LIMIT 1"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                     Which frequent flyer program has the most airlines?                                      |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | SELECT ?uri WHERE { ?airline <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Airline> . ?airl |
|           | ine <http://dbpedia.org/property/frequentFlyer> ?uri. } GROUP BY ?uri ORDER BY DESC(COUNT(DISTINCT ?airline)) OFFSET 0 LIMIT |
|           |                                                               1                                                              |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |

In [76]:
# From QALD
q = 'Which European countries have a constitutional monarchy?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> PREFIX dbr: <http://dbpedia.org/resource/> SELECT ?uri WHERE { ?uri dct:subject dbc:Countries_in_Europe ; dbo:governmentType dbr:Constitutional_monarchy }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                   Which European countries have a constitutional monarchy?                                   |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/ |
|           | Category:> PREFIX dbr: <http://dbpedia.org/resource/> SELECT ?uri WHERE { ?uri dct:subject dbc:Countries_in_Europe ; dbo:gov |
|           |                                          ernmentType dbr:Constitutional_monarchy }                                           |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |

In [77]:
# From QALD
q = 'Which countries have places with more than two caves?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> SELECT DISTINCT ?uri WHERE { ?cave rdf:type dbo:Cave ; dbo:location ?uri . ?uri rdf:type dbo:Country } GROUP BY ?uri HAVING ( COUNT(?cave) > 2 )"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                    Which countries have places with more than two caves?                                     |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> SELECT DISTINCT ?uri WH |
|           |  ERE { ?cave rdf:type dbo:Cave ; dbo:location ?uri . ?uri rdf:type dbo:Country } GROUP BY ?uri HAVING ( COUNT(?cave) > 2 )   |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |                       select ?answer where [ wd:horse wdt:place_of_birth ?X . ?X wdt:country ?answer]                        |
+-----------+

In [78]:
# From QALD
q = 'Which airports are located in California, USA?'
target = "SELECT DISTINCT ?uri WHERE { ?uri a <http://dbpedia.org/ontology/Airport> { ?uri <http://dbpedia.org/ontology/location> <http://dbpedia.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/city> <http://dbpedia.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/city> ?city . ?city <http://dbpedia.org/ontology/isPartOf> <http://dbpedia.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/operator> <http://dbpedia.org/resource/California> } }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                        Which airports are located in California, USA?                                        |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | SELECT DISTINCT ?uri WHERE { ?uri a <http://dbpedia.org/ontology/Airport> { ?uri <http://dbpedia.org/ontology/location> <htt |
|           | p://dbpedia.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/city> <http://dbpedia.org/resource/Californ |
|           | ia> } UNION { ?uri <http://dbpedia.org/ontology/city> ?city . ?city <http://dbpedia.org/ontology/isPartOf> <http://dbpedia.o |
|           |  rg/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/operator> <http://dbpedia.org/resource/California> } }  |
+-----------+

In [79]:
# From QALD
q = "What are the nicknames of San Francisco?"
target = "SELECT DISTINCT ?string WHERE { res:San_Francisco foaf:nick ?string }"
pretty_translate(target, q)

+-----------+--------------------------------------------------------------------------------------------------+
| QUESTION  |                             What are the nicknames of San Francisco?                             |
+-----------+--------------------------------------------------------------------------------------------------+
|  Target   |              SELECT DISTINCT ?string WHERE { res:San_Francisco foaf:nick ?string }               |
+-----------+--------------------------------------------------------------------------------------------------+
| RawRESULT | select distinct ?obj where [ wd:san_france wdt:named_after ?obj . ?obj wdt:instance_of wd:name ] |
+-----------+--------------------------------------------------------------------------------------------------+
|  RESULT   |       select distinct ?obj where { wd:san_france wdt:P138 ?obj . ?obj wdt:P31 wd:Q82799 }        |
+-----------+-----------------------------------------------------------------------------------

In [80]:
# From QALD
q = "What is Angela Merkel’s birth name?"
target = "SELECT DISTINCT ?string WHERE { res:Angela_Merkel dbp:birthName ?string }"
pretty_translate(target, q)

+-----------+---------------------------------------------------------------------------------------+
| QUESTION  |                          What is Angela Merkel’s birth name?                          |
+-----------+---------------------------------------------------------------------------------------+
|  Target   |       SELECT DISTINCT ?string WHERE { res:Angela_Merkel dbp:birthName ?string }       |
+-----------+---------------------------------------------------------------------------------------+
| RawRESULT | select ?answer where [ wd:angela_familia wdt:born_of ?X . ?X wdt:named_after ?answer] |
+-----------+---------------------------------------------------------------------------------------+
|  RESULT   |    select ?answer where { wd:angela_familia wdt:born_of ?X . ?X wdt:P138 ?answer}     |
+-----------+---------------------------------------------------------------------------------------+


In [81]:
# From QALD
q = "Who is the mayor of Berlin?"
target = "SELECT DISTINCT ?uri WHERE { res:Berlin dbp:leader ?uri }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------+
| QUESTION  |                                     Who is the mayor of Berlin?                                      |
+-----------+------------------------------------------------------------------------------------------------------+
|  Target   |                      SELECT DISTINCT ?uri WHERE { res:Berlin dbp:leader ?uri }                       |
+-----------+------------------------------------------------------------------------------------------------------+
| RawRESULT | select distinct ?obj where [ wd:berlin wdt:head_of_government ?obj . ?obj wdt:instance_of wd:human ] |
+-----------+------------------------------------------------------------------------------------------------------+
|  RESULT   |                select distinct ?obj where { wd:Q64 wdt:P6 ?obj . ?obj wdt:P31 wd:Q5 }                |
+-----------+---------------------------------------------------

In [82]:
# From QALD
q = "Which software has been published by Mean Hamster Software?"
target = "SELECT DISTINCT ?uri WHERE { ?uri rdf:type onto:Software { ?uri prop:publisher \"Mean Hamster Software\"@en } UNION { ?uri onto:publisher res:Mean_Hamster_Software } }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                 Which software has been published by Mean Hamster Software?                                  |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | SELECT DISTINCT ?uri WHERE { ?uri rdf:type onto:Software { ?uri prop:publisher "Mean Hamster Software"@en } UNION { ?uri ont |
|           |                                          o:publisher res:Mean_Hamster_Software } }                                           |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |             select ?answer where [ wd:moyen_hamster_software wdt:published_by ?X . ?X wdt:published_by ?answer]              |
+-----------+

In [83]:
# From QALD
q = "Which country was Bill Gates born in?"
target = "SELECT DISTINCT ?country WHERE { { dbr:Bill_Gates dbo:birthPlace ?birthPlace . ?birthPlace dbo:country ?country } UNION { dbr:Bill_Gates dbo:birthPlace ?birthPlace . ?birthPlace dbo:isPartOf ?place . ?place dbo:country ?country } }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                            Which country was Bill Gates born in?                                             |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | SELECT DISTINCT ?country WHERE { { dbr:Bill_Gates dbo:birthPlace ?birthPlace . ?birthPlace dbo:country ?country } UNION { db |
|           |         r:Bill_Gates dbo:birthPlace ?birthPlace . ?birthPlace dbo:isPartOf ?place . ?place dbo:country ?country } }          |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |                    select ?answer where [ wd:billy_gates wdt:place_of_birth ?X . ?X wdt:country ?answer]                     |
+-----------+

In [84]:
# From QALD
q = "How many grand-children did Jacques Cousteau have?"
target = "SELECT COUNT(DISTINCT ?y AS ?y) WHERE { <http://dbpedia.org/resource/Jacques_Cousteau> <http://dbpedia.org/ontology/child> ?x . ?x <http://dbpedia.org/ontology/child> ?y . }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                      How many grand-children did Jacques Cousteau have?                                      |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | SELECT COUNT(DISTINCT ?y AS ?y) WHERE { <http://dbpedia.org/resource/Jacques_Cousteau> <http://dbpedia.org/ontology/child> ? |
|           |                                      x . ?x <http://dbpedia.org/ontology/child> ?y . }                                       |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |                         select (COUNT(?sub) AS ?value ) [ ?sub wdt:grand-child wd:jacques_cousteau ]                         |
+-----------+

In [85]:
# From QALD
q = "Give me all professional skateboarders from Sweden."
target = "SELECT DISTINCT ?uri WHERE { ?uri dbo:occupation dbr:Skateboarder { ?uri dbo:birthPlace dbr:Sweden } UNION { ?uri dbo:birthPlace ?place . ?place dbo:country dbr:Sweden } }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                     Give me all professional skateboarders from Sweden.                                      |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | SELECT DISTINCT ?uri WHERE { ?uri dbo:occupation dbr:Skateboarder { ?uri dbo:birthPlace dbr:Sweden } UNION { ?uri dbo:birthP |
|           |                                       lace ?place . ?place dbo:country dbr:Sweden } }                                        |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |                select ?answer where [ wd:sweden wdt:professional_skateboarder ?X . ?X wdt:given_name ?answer]                |
+-----------+

In [86]:
# From QALD
q = "Which monarchs of the United Kingdom were married to a German?"
target = "SELECT DISTINCT ?uri WHERE { ?uri rdf:type yago:WikicatMonarchsOfTheUnitedKingdom ; dbo:spouse ?spouse . ?spouse dbo:birthPlace res:Germany }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                Which monarchs of the United Kingdom were married to a German?                                |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | SELECT DISTINCT ?uri WHERE { ?uri rdf:type yago:WikicatMonarchsOfTheUnitedKingdom ; dbo:spouse ?spouse . ?spouse dbo:birthPl |
|           |                                                      ace res:Germany }                                                       |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |              select distinct ?obj where [ wd:united_kingdom wdt:spouse ?obj . ?obj wdt:instance_of wd:monarch ]              |
+-----------+

In [87]:
# From QALD
q = "Give me all Argentine films."
target = "SELECT DISTINCT ?uri WHERE { { ?uri rdf:type yago:ArgentineFilms } UNION { ?uri rdf:type dbo:Film { ?uri dbo:country res:Argentina } UNION { ?uri dbp:country \"Argentina\"@en } } }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                                 Give me all Argentine films.                                                 |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
|  Target   | SELECT DISTINCT ?uri WHERE { { ?uri rdf:type yago:ArgentineFilms } UNION { ?uri rdf:type dbo:Film { ?uri dbo:country res:Arg |
|           |                                    entina } UNION { ?uri dbp:country "Argentina"@en } } }                                    |
+-----------+------------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |                   select ?answer where [ wd:argentina wdt:film_director ?X . ?X wdt:film_director ?answer]                   |
+-----------+

In [88]:
# From QALD
q = "How did Michael Jackson die?"
target = "SELECT DISTINCT ?s WHERE { <http://dbpedia.org/resource/Michael_Jackson> <http://dbpedia.org/property/deathCause> ?s }"
pretty_translate(target, q)

+-----------+------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                              How did Michael Jackson die?                                              |
+-----------+------------------------------------------------------------------------------------------------------------------------+
|  Target   | SELECT DISTINCT ?s WHERE { <http://dbpedia.org/resource/Michael_Jackson> <http://dbpedia.org/property/deathCause> ?s } |
+-----------+------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |  select ?value where [ wd:michael_jennifer p:death ?s . ?s ps:death wd:michael_jennifer . ?s pq:point_in_time ?value]  |
+-----------+------------------------------------------------------------------------------------------------------------------------+
|  RESULT   |       select ?value where { wd:michael_je

In [89]:
# From QALD
q = "Where did Michael Jackson died?"
target = "."
pretty_translate(target, q)

+-----------+---------------------------------------------------------------------------------------------------+
| QUESTION  |                                  Where did Michael Jackson died?                                  |
+-----------+---------------------------------------------------------------------------------------------------+
|  Target   |                                                 .                                                 |
+-----------+---------------------------------------------------------------------------------------------------+
| RawRESULT | select ?answer where [ wd:michael_jennifer wdt:place_of_death ?X . ?X wdt:place_of_death ?answer] |
+-----------+---------------------------------------------------------------------------------------------------+
|  RESULT   |            select ?answer where { wd:michael_jennifer wdt:P20 ?X . ?X wdt:P20 ?answer}            |
+-----------+---------------------------------------------------------------------------

In [90]:
# From QALD
q = "Which classes does the Millepede belong to?"
target = "SELECT DISTINCT ?String WHERE { res:Millipede dbp:taxon ?String }"
pretty_translate(target, q)

+-----------+----------------------------------------------------------------------------------------------------+
| QUESTION  |                            Which classes does the Millepede belong to?                             |
+-----------+----------------------------------------------------------------------------------------------------+
|  Target   |                 SELECT DISTINCT ?String WHERE { res:Millipede dbp:taxon ?String }                  |
+-----------+----------------------------------------------------------------------------------------------------+
| RawRESULT | select ?answer where [ wd:millepede wdt:member_of_the_millepede ?X . ?X wdt:class_of_work ?answer] |
+-----------+----------------------------------------------------------------------------------------------------+
|  RESULT   | select ?answer where { wd:millepede wdt:member_of_the_millepede ?X . ?X wdt:class_of_work ?answer} |
+-----------+-------------------------------------------------------------------

In [91]:
# From QALD
q = "Which classes does the Millepede belong to?"
target = "SELECT DISTINCT ?String WHERE { res:Millipede dbp:taxon ?String }"
pretty_translate(target, q)

+-----------+----------------------------------------------------------------------------------------------------+
| QUESTION  |                            Which classes does the Millepede belong to?                             |
+-----------+----------------------------------------------------------------------------------------------------+
|  Target   |                 SELECT DISTINCT ?String WHERE { res:Millipede dbp:taxon ?String }                  |
+-----------+----------------------------------------------------------------------------------------------------+
| RawRESULT | select ?answer where [ wd:millepede wdt:member_of_the_millepede ?X . ?X wdt:class_of_work ?answer] |
+-----------+----------------------------------------------------------------------------------------------------+
|  RESULT   | select ?answer where { wd:millepede wdt:member_of_the_millepede ?X . ?X wdt:class_of_work ?answer} |
+-----------+-------------------------------------------------------------------